In [1]:
#check if mace_run_train is available
#!whre mace_run_train

In [11]:
#import libraries
import numpy as np
from ase.io import read, write
from ase import Atoms
import os
os.makedirs('config', exist_ok=True)
import torch

In [3]:
db = read('251006_training_data.xyz',':')

In [1]:
#len(db)

In [5]:
#dividing the dataframe to get 2 sets, one for training and the other for testing
write('train.xyz', db[:250])
write('test.xyz', db[250:])

In [23]:
%%writefile config_mace_3bpa.yml
model: "MACE"
num_channels: 32
max_L: 0
r_max: 4.0
name: "mace_test"
train_file: "train.xyz"
valid_fraction: 0.1
test_file: "test.xyz"
energy_key: "energy"
forces_key: "forces"
E0: "average"
batch_size: 20
max_num_epochs: 10
device: cpu
seed: 123

Overwriting config_mace_3bpa.yml


In [24]:
import warnings
warnings.filterwarnings("ignore")
from mace.cli.run_train import main as mace_run_train_main
import sys
import logging

def train_mace(config_file_path):
    logging.getLogger().handlers.clear()
    sys.argv = ["program", "--config", config_file_path]  # Simulates command line
    mace_run_train_main()  # Calls the main function directly

train_mace("config_mace_3bpa.yml")

2025-10-14 01:11:10.370 INFO: ===========VERIFYING SETTINGS===========
2025-10-14 01:11:10.371 INFO: MACE version: 0.3.14
2025-10-14 01:11:10.372 INFO: Using CPU
2025-10-14 01:11:10.414 INFO: ===========LOADING INPUT DATA===========
2025-10-14 01:11:10.415 INFO: Using heads: ['Default']
2025-10-14 01:11:10.416 INFO: Using the key specifications to parse data:
2025-10-14 01:11:10.416 INFO: Default: KeySpecification(info_keys={'energy': 'energy', 'stress': 'REF_stress', 'virials': 'REF_virials', 'dipole': 'dipole', 'head': 'head', 'elec_temp': 'elec_temp', 'total_charge': 'total_charge', 'polarizability': 'polarizability', 'total_spin': 'total_spin'}, arrays_keys={'forces': 'forces', 'charges': 'REF_charges'})
2025-10-14 01:11:10.416 INFO: =============    Processing head Default     ===========
2025-10-14 01:11:10.467 WARNING: Since ASE version 3.23.0b1, using energy_key 'energy' is no longer safe when communicating between MACE and ASE. We recommend using a different key, rewriting 'en

In [8]:
from ase import units #physical constants
from ase.md.langevin import Langevin #langevin dinamics integrator
from ase.io import read, write # read/write chemical structure files
import numpy as np 
import time #timing performance

from mace.calculators import MACECalculator # import the machine learning potential -> gives acces to mace as a "calculator" that can predict energies and forces

calculator = MACECalculator(model_paths='mace_test.model', device='cpu') #loads the trained MACe model
init_conf = read('test.xyz', '0') # loads the starting structure for MD, loads a singe molecular configuration to start the simulation
init_conf.set_calculator(calculator) # connect the ML calculator to the atoms (en teoria se puede usar init_conf.cal = calculator pero no lo he probado)

dyn = Langevin(init_conf, 0.5*units.fs, temperature_K=310, friction=5e-3) #uses langevin dynamics (constant temperature)
#defines a function to save simulation snapshots
def write_frame():
        dyn.atoms.write('md_3bpa.xyz', append=True)
dyn.attach(write_frame, interval=50) #indicates the simulator when to save frames
dyn.run(100) #execute the molecular dynamics, each step calculates forces using mace, updates atom positions and velocities, applies thermostat and saves trajectory frame every 50 steps
print("MD finished!")

2025-10-14 00:52:04.599 INFO: Using CPU
Using head Default out of ['Default']
No dtype selected, switching to float64 to match model dtype.
MD finished!


In [ ]:
from ase.io import read

trajectory = read('md_3bpa.xyz', index=':')
print(f"trajectory contains {len(trajectory)} frames")
print(f"each frame has {len(trajectory[0])} atoms")

# Print energies from the trajectory
print("\nEnergies from MD simulation:")
for i, atoms in enumerate(trajectory):
    if hasattr(atoms, 'get_potential_energy'):
        energy = atoms.get_potential_energy()
        print(f"Frame {i}: Energy = {energy:.4f} eV")

Trajectory contains 9 frames
Each frame has 10 atoms

Energies from MD simulation:
Frame 0: Energy = -70.2193 eV
Frame 1: Energy = -71.7766 eV
Frame 2: Energy = -72.1329 eV
Frame 3: Energy = -70.2193 eV
Frame 4: Energy = -71.7766 eV
Frame 5: Energy = -72.1329 eV
Frame 6: Energy = -73.0844 eV
Frame 7: Energy = -74.6088 eV
Frame 8: Energy = -75.0692 eV


In [12]:
# Check that MPS is available
if torch.backends.mps.is_available():
    print('available')
else:
    print("not available")

available


In [13]:
mps_device = torch.device("mps")

# Create a Tensor directly on the mps device
x = torch.ones(5, device=mps_device, dtype=torch.float32)
    # Or
#x = torch.ones(5, device="mps")

    # Any operation happens on the GPU
y = x * 2
y

tensor([2., 2., 2., 2., 2.], device='mps:0', dtype=torch.float32)

In [25]:
%%writefile config_mace_3bpa_mps.yml
model: "MACE"
num_channels: 32
max_L: 0
r_max: 4.0
name: "mace_test"
train_file: "train.xyz"
valid_fraction: 0.1
test_file: "test.xyz"
energy_key: "energy"
forces_key: "forces"
E0: "average"
batch_size: 10
max_num_epochs: 5
device: mps
seed: 123
default_dtype: "float32"

Overwriting config_mace_3bpa_mps.yml


In [26]:
import warnings
warnings.filterwarnings("ignore")
from mace.cli.run_train import main as mace_run_train_main
import sys
import logging

def train_mace(config_file_path):
    logging.getLogger().handlers.clear()
    sys.argv = ["program", "--config", config_file_path]  # Simulates command line
    mace_run_train_main()  # Calls the main function directly

train_mace("config_mace_3bpa_mps.yml")

2025-10-14 01:12:40.217 INFO: ===========VERIFYING SETTINGS===========
2025-10-14 01:12:40.218 INFO: MACE version: 0.3.14
2025-10-14 01:12:40.219 INFO: Using MPS GPU acceleration
2025-10-14 01:12:40.261 INFO: ===========LOADING INPUT DATA===========
2025-10-14 01:12:40.262 INFO: Using heads: ['Default']
2025-10-14 01:12:40.263 INFO: Using the key specifications to parse data:
2025-10-14 01:12:40.263 INFO: Default: KeySpecification(info_keys={'energy': 'energy', 'stress': 'REF_stress', 'virials': 'REF_virials', 'dipole': 'dipole', 'head': 'head', 'elec_temp': 'elec_temp', 'total_charge': 'total_charge', 'polarizability': 'polarizability', 'total_spin': 'total_spin'}, arrays_keys={'forces': 'forces', 'charges': 'REF_charges'})
2025-10-14 01:12:40.264 INFO: =============    Processing head Default     ===========
2025-10-14 01:12:40.315 WARNING: Since ASE version 3.23.0b1, using energy_key 'energy' is no longer safe when communicating between MACE and ASE. We recommend using a different k

TypeError: Cannot convert a MPS Tensor to float64 dtype as the MPS framework doesn't support float64. Please use float32 instead.